In [ ]:
import os
import re
import glob 
import matplotlib.pyplot as plt
import numpy as np
from os.path import basename
from multiprocessing import Pool
import sys
import numpy
import librosa
import pickle
import acousticFeatures
from typing import Dict, Tuple, List

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}




#Define class
class Input:
    
    def __init__(self, code=None, spectrogram=None, acoustic_features=None, transcript=None):
        self.__code = code
        self.__spectrogram = spectrogram
        self.__acoustic_features = acoustic_features
        self.__transcript = transcript
    
    def get_code(self):
        return self.__code
            
    def get_spectrogram(self):
        return self.__spectrogram
    
    def get_acoustic_features(self):
        return self.__acoustic_features
    
    def get_transcript(self):
        return self.__transcript
    
    def set_transcript(self, transcript):
        self.__transcript = transcript
    
    
    
    
class Output:
    def __init__(self, code, duration, annotated_categories, chosen_category, attributes):
        self.__code = code # file name
        self.__duration = duration #  duration [t1-t2]
        self.__annotated_categories= annotated_categories # array of different category evaluation of annotator
        self.__chosen_category = chosen_category  # final category which most of annotators choose
        self.__attributes = attributes # valance, arouse, dominance of emotion
        
    
    def get_code(self):
        return self.__code

    def get_duration(self):
        return self.__duration
    
    def get_annotated_categories(self):
        return self.__annotated_categories
    
    def get_chosen_category(self):
        return self.__chosen_category
    
    def get_attributes(self):
        return self.__attributes
        
        
    def get_consitency(self):
        """Check if most of annotators have the same evaluation.
        """
        counts = np.unique(self.get_annotated_categories(), return_counts=True)[1]
        consitent_degree = np.max(counts)/np.sum(counts)
        return consitent_degree
            
    

        
#Function for getting input vector and corresponding output      
def parallel_task(d0, d1, d2):
    print("task...")
    
    def get_transcription(file: str) -> Dict[str, str] :
        """ Get all transriptions in file

            :param file: It is the name(code) of the file
            :return: Dict. Keys are name of audio files. Values are their transcrips
        """
        dicts = {} # Dict containing the result 
        with open(file) as f:
            for line in f:
                parts = line.split(' ', maxsplit=2) # Example of one line in file: Name_file [t1-t2]: Transcript.
                try:
                    dicts[parts[0]] = parts[2]
                except:
                    print("Error caused by: ", parts)
                    print("In file:", file)
                    dicts[parts[0]] = ""

        return dicts
    
    
    
    def parseInput(dir):
        """Each input diectory contains many file
           This fucntion will walk through all valid 'wav'files in this directory and get features like spectrogram, acoustic
           Transcript will insert into input later.
        """
        
        dicts = {} 
        for f in os.listdir(dir):
            if not f.startswith(".") and os.path.splitext(f)[1] == ".wav":
                # Get file name
                code = os.path.splitext(f)[0] 
                
                # Get file path
                file_path = "/".join([dir, code]) + ".wav"
                
                # Create histogram
                y, sr = librosa.load(file_path)
                spectrogram = librosa.stft(y)
                spectrogram = np.abs(spectrogram)
                
                # Create acoustic features
                acoustic_features = acousticFeatures.getAllFeatures(file_path)
                
                dicts[code] = Input(code, spectrogram, acoustic_features, transcript=None)

        return dicts
    
    
    def parseOutput(file):
        """Each output file contains label of many diffrent 'wav' file.
           This function will parse content of text file using 'regrex'. Then turn it into label
        """
        
        
        dict_namefile_output = {}
        
        # Open file to get all contents excepts the first line.
        f = open(file, 'r')
        content = ""
        index = 0
        for line in f:
            index = index + 1
            if index == 1:
                continue
            content  = content + line

        # Find all matched patterns in the content
        ps = re.findall(r'\[.*?\)\n\n', content, re.DOTALL)

        # Parse each matched pattern into  'Output' object
        try:
            for p in ps:
                ls = p.split("\n")
                ls = list(filter(lambda x: len(x) > 0 ,ls))

                # Split elements of the first line which looks like : 
                # [147.0300 - 151.7101]	Ses01F_impro02_M012	neu	[2.5000, 2.0000, 2.0000]
                ele_line0 = re.search(r'(\[.*?\])(\s)(.*?)(\s)(.*?)(\s)(\[.*?\])', ls[0]).groups()

                # Split time components which looks like:
                # [147.0300 - 151.7101]
                time_dur = ele_line0[0]
                ele_time_dur = re.findall(r"[-+]?\d*\.\d+|\d+", time_dur)
                ele_time_dur = [float(x) for x in ele_time_dur]

                # Get code and category_origin which looks like:
                # Code: Ses01F_impro02_M012
                # Category_origin: neu
                code = ele_line0[2]
                chosen_category = ele_line0[4]

                # Split attribute components which looks like:
                # [2.5000, 2.0000, 2.0000]
                attribute = ele_line0[6]
                ele_attribute = re.findall(r"[-+]?\d*\.\d+|\d+", attribute)
                ele_attribute = [float(x) for x in ele_attribute]

                # Get categorial_evaluation:
                lines_categorical = list(filter(lambda x : x[0] == 'C', ls))
                rex = re.compile(r'C.*?:(\s)(.*?)(\s)\(.*?\)')

                annotated_categories = []
                for l in lines_categorical:
                    elements = rex.search(l).groups()
                    cat = elements[1]
                    cat = cat.split(";")
                    cat = map(lambda x: x.lstrip(), cat)
                    cat = list(filter(lambda x: len(x)>0, cat))
                    annotated_categories.extend(cat)


                annotated_categories = np.array(annotated_categories)
                                
                

                # Make dict {name_file : parsed_output}
                dict_namefile_output[code] = Output(code, ele_time_dur, annotated_categories, chosen_category, ele_attribute)
            return dict_namefile_output
        except Exception as e:
            print(e)


    ### Parse input and output files and get input and output as vector
    dicts_in = parseInput(d0)
    dicts_out = parseOutput(d1)
    dicts_transcript = get_transcription(d2)
    in_out = []
    
    keys = list(dicts_in.keys())
    for key in keys:
        # Insert transcript into Input object
        dicts_in[key].set_transcript(dicts_transcript[key])
        
        # Make a tuple (Input, Output)
        in_out.append((dicts_in[key], dicts_out[key]))
            
    return in_out
    
    
def createInput_Output():
    ### Get directories of input and output
    DATA_DIR = "auditary_emotion_recognition/IEMOCAP_full_release"
    NUM_SESSION = 5
    input_output = []
    for i in range (1, NUM_SESSION + 1):
        name_session = "Session" + str(i)
        root_dir_of_wav = DATA_DIR + "/" + name_session + "/sentences" + "/wav"
        root_dir_of_labels = DATA_DIR + "/" + name_session + "/dialog" + "/EmoEvaluation"
        root_dir_of_transcripts = DATA_DIR + "/" + name_session + "/dialog" + "/transcriptions"

        for x in os.walk(root_dir_of_wav):
            if(x[0] == root_dir_of_wav):
                dirs_of_wav = x[1]
                index = -1
            else:
                index = index + 1
                input_output.append((x[0], 
                                     root_dir_of_labels + "/" + dirs_of_wav[index] + ".txt",
                                     root_dir_of_transcripts + "/" + dirs_of_wav[index] + ".txt"
                                   ))
                
    
    ds = input_output
    in_out = []
    
    # Multi processing
    with Pool(processes=30) as pool:
         in_out = pool.starmap(parallel_task, ds)
    
    # Append all the result which returned by multi proccessing.
    r = []
    for e in in_out:
        r = r + e
    
    input = [x[0] for x in r]
    out = [x[1] for x in r]
    print("Finished creating input output into txt file")
    print("len input and output:", len(input), ", ", len(out))
    
    return (input, out)
 




# Create input, output 
input, output = createInput_Output()
input = np.array(input)
output = np.array(output)
  
# Define file handler
filehandlerInput = open('processed-data/input.obj', 'wb')
filehandlerOutput = open('processed-data/output.obj', 'wb')

# Dump to files
pickle.dump(input, filehandlerInput)
pickle.dump(output, filehandlerOutput)
print("Finish write processed data (input, output) to file!!!")

In [ ]:
filehandlerInput = open('processed-data/input.obj', 'rb')
filehandlerOutput = open('processed-data/output.obj', 'rb')

# Dump to files
x = pickle.load(filehandlerInput)
y = pickle.load( filehandlerOutput)

for i in y:
    print("category: {}, VAD: {}, consitent: {}".format(i.get_chosen_category(), i.get_attributes(), i.isConsitent()))